# MSDM5051 Tutorial 5 - Binary Search Tree

## Content

1. Basic Binary Search Tree Operations
2. Self-balancing BST - AVL Tree and Red Black Tree
3. Practices

---
# 1. Binary Search Tree Operations

Recall the construction rule of a binary search tree (BST):

- All the data values in its left sub-tree are smaller than the data value in itself.
- All the data values in its right sub-tree are larger than the data value in itself.

We can see that BST is essentially a decision tree. Here we start the code with the skeleton of a binary tree.

In [1]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        
class BST:
    def __init__(self, root = None):    # A BST can be declared via defining the root node
        self.root = root
        
    ###################################################################################################################################
    def print_tree(self, prop=''):      # Roughly print the tree node for checking. This is a simplified version of depth-first Search.
                                        # can have prop='height' in AVL tree and prop='color' in Red Black tree
        
        if self.root:
            tree_dict = {self.root:(0,0)}   # record: 1. height of the node (start from 0); 2. heap index (because calculation is easy)
            
            def print_DFS(node):
                height_L = 1
                height_R = 1
                if node.left:
                    tree_dict[node.left] = (tree_dict[node][0]+1, tree_dict[node][1]*2+1)     
                    height_L = print_DFS(node.left)+1
                if node.right:
                    tree_dict[node.right] = (tree_dict[node][0]+1, tree_dict[node][1]*2+2)
                    height_R = print_DFS(node.right)+1
                return max(height_L, height_R)
            
            height = print_DFS(self.root)
        
            tree_disp = []
            for h in range(height):
                tree_disp.append([None]*2**h)              # use max height to create a empty tree made of None array

            if getattr(self.root, prop, None) is not None:
                # if prop is a valid attribute of the node
                # substitute the node's data and the corresponding attribute in the corresponding position in the tree
                for node, pos in tree_dict.items():
                    tree_disp[pos[0]][pos[1]+1-2**pos[0]] = (node.data, getattr(node, prop))

            else:
                # otherwise only substitute the node's data in the corresponding position in the tree
                for node, pos in tree_dict.items():
                    tree_disp[pos[0]][pos[1]+1-2**pos[0]] = node.data

            for i in tree_disp:
                print(i)
                
        else:
            print("the tree is empty")
            

## 1.1. Construction / Insertion

Construction from array / Insertion is as simple as passing a value from root to a leaf position via a series of comparisons:

1. Make the first element the root of the BST.
2. For 2nd elements and onwards, compare its value with the root to see if it should be pass to the left or right branch:
    - **Stopping condition:** If the branch is empty, place down the element.
    - **Inductive step:** If the branch is not empty, continue comparison with the child node at that branch.

<figure style="text-align: center">

    <figcaption> <b>Fig. 1</b> Converting array to BST by passing elements through a decision tree.</figcaption>
</figure>

In [2]:
# When declared using an array
def insert_from_array(self, array):
    for data in array:
        self.root = self.insert(Node(data), self.root)    # insertion always begin with the root

# Insertion via recursion
def insert(self, new_node, curr):

    if curr is None:       # curr is an empty positon, so possible to insert
        return new_node    # tell curr's parent to link to this new_node

    elif new_node.data < curr.data:                    # go to left branch
        curr.left = self.insert(new_node, curr.left)   # curr asks its left branch if it should make new_node its new left child

    else:                                              # go to right branch
        curr.right = self.insert(new_node, curr.right) # curr asks its right branch if it should make new_node its new right child

    return curr                 # in the above two cases, after previous recursion returns, since curr already exists
                                # tell curr's parent that it should still link to curr
    
BST.insert_from_array = insert_from_array
BST.insert = insert

In [3]:
tree = BST()
a = [20,11,29]
b = [33,5,17,1,8]

tree.insert_from_array(a)
tree.insert_from_array(b)
tree.print_tree()

[20]
[11, 29]
[5, 17, None, 33]
[1, 8, None, None, None, None, None, None]


## 1.2. Search

Searching in BST is almost the same as insertion as it just needs to run down the decision tree. Here is a code example.

In [4]:
# input target value, return the first node object that has the same value if found. return None if not found.
def search_data(self, target_value):
    return self.search(target_value, self.root)
    
# Search via recursion
def search(self, target_value, curr):

    if (curr is None) or (curr.data == target_value):    # if the node is found, return this node, or
        return curr                                      # if reaching a None node, return None
        
    elif target_value < curr.data:                       # go to left branch
        return self.search(target_value, curr.left)      # ask the left branch to continue searching

    else:                                                # go to right branch
        return self.search(target_value, curr.right)     # ask the right branch to continue searching
    
BST.search_data = search_data
BST.search = search

In [5]:
print(tree.search_data(17), tree.search_data(17).data)
print(tree.search_data(4))

<__main__.Node object at 0x0000021EA3981910> 17
None


## 1.3. Deletion

Deletion in BST is separated into 3 cases depending on how many children the target node has:

1. **No children** - You can un-link it directly.

2. **1 child** - Link its parents to its child.

3. **2 children** - Replace it by the node whose value is the closest to it. 

Found that node in case 3 is in fact simple because of the decision tree structure. For example, one can find such node by going to the right branch first, then find the left-most grandchild. (i.e. 1 step right, then go to left as far as you can.) 

**Note 1:** Here chooses the right branch is chosen because it includes the *equal to*. In theory, we can also use the right-most grandchild on the left branch, but the whole algorithm will be mirrored.

<figure style="text-align: center">

    <figcaption> <b>Fig. 2</b> 3 cases in BST deletion.</figcaption>
</figure>

**Note 2**: In the 2 children case, we can simply do the 3 re-links as shown. However, we can also observer that:
- The left-most descendent has at most 1 child, because its left child must be `None`, or otherwise it is not the left-most.
- To taking out the left-most descendent, we have to first "delete" it from its original position. This deletion must be either case 1 or 2.

So an alternative operation is that we first run a new recursion from the deleted node to the left-most descendent, then continue the original recursive deletion from root to the deleted node. This is a bit slower, but the codes are simplier when we are dealing with the self-balancing trees. (See the codes in the self-balancing tree sections.)

In [6]:
## input target value, delete the first node object that has the same value if it is found.
def delete_data(self, target_value):
    self.root = self.delete(target_value, self.root)


# for finding the leftmost descendent in case 3
def leftmost_descendant(self, node):
    while node.left:
        node = node.left
    return node              # the returned node is the leftmost descendent


def delete(self, target_value, curr):
    
    if curr is None:    # the case when no node contain the target value
        print('no node found')
        return None 
    
    # search for the node that contains the target value
    if target_value == curr.data:    # when curr is the matching node

        # case 1: curr has no child
        if curr.left is None and curr.right is None:
            return None    # tell curr's parent it should link to None

        
        # case 2: curr has 1 child
        if curr.right is None:           # if it is a left child
            return curr.left    # tell curr's parent to link to its left branch
        elif curr.left is None:
            return curr.right   # tell curr's parent to link to its right branch

        
        # case 3: curr has 2 children. This is the direct re-linking method
        desc = self.leftmost_descendant(curr.right)
        
        desc.left = curr.left               # the left hand side always re-link
        
        if curr.right != desc:              # if curr's right child is already the leftmost descendent, no need to re-link right side
                                            # simply replace the deleted note by its right child
            curr.right.left = desc.right    # otherwise do the two re-link on the right hand side
            desc.right = curr.right
        
        return desc                         # tell curr's parent to link to this leftmost descendent

    
    elif target_value < curr.data: 
        curr.left = self.delete(target_value, curr.left)    # curr asking its left branch if it should link to new a left child

    else:
        curr.right = self.delete(target_value, curr.right)  # curr asking its right branch if it should link to new a right child

    return curr                                             # after previous recursion returns, since curr is not  
                                                            # a matching node, curr's parent should still link to curr

    
BST.delete_data = delete_data
BST.delete = delete
BST.leftmost_descendant = leftmost_descendant

In [7]:
# Case 1
tree = BST()
tree.insert_from_array([20,11,29,5,17,24,33,1,8,27])
tree.delete_data(8)
tree.print_tree()

[20]
[11, 29]
[5, 17, 24, 33]
[1, None, None, None, None, 27, None, None]


In [8]:
# Case 2
tree = BST()
tree.insert_from_array([20,11,29,5,17,24,33,1,8,27])
tree.delete_data(24)
tree.print_tree()

[20]
[11, 29]
[5, 17, 27, 33]
[1, 8, None, None, None, None, None, None]


In [9]:
# Case 3
tree = BST()
tree.insert_from_array([20,11,29,5,17,24,33,1,8,27,19,14])
tree.delete_data(11)
tree.print_tree()
tree.root.left.left.data

[20]
[14, 29]
[5, 17, 24, 33]
[1, 8, None, 19, None, 27, None, None]


5

---
# 2. Self-balancing BST

The 2 self-balancing BST introduced in the lecture are:

- AVL Tree
- Red Black Tree

All 3 kinds of operations: insert, search, delete start the same as an ordinary BST, however additional steps are required to re-balance the tree after inserting/deleting nodes. 

## 2.1. Left/Right rotate

The re-balancing steps of both kinds of tree are composed of a series of two operations - `rotate_left()` and `rotate_right()`. What they do are quite self-explaining with this animation from wiki:

<figure style="text-align: center">
<img src="https://upload.wikimedia.org/wikipedia/commons/f/f2/Binary_Tree_Rotation_%28animated%29.gif" alt="left right rotate" style="width:20%;min-width:200px" />
    <figcaption> <b>Fig. 3</b> Left & right rotate in BST. Retrieved from <a href="https://en.wikipedia.org/wiki/File:Binary_Tree_Rotation_(animated).gif">https://en.wikipedia.org/wiki/File:Binary_Tree_Rotation_(animated).gif</a>.</figcaption>
</figure>

The idea of these two functions are pretty simple if you view the link between L node and R node as a balance that lifts up the taller subtree, with the middle subtree as a pivot -

- if the left subtree of L is taller, `rotate_right()` to lift it up.
- if the right subtree of R is taller, `rotate_left()` to lift it up.

And here is an example how you can code these two operations:

In [10]:
def rotate_left(self, L):
    
    # define the original parent-child relation
    R = L.right
    mid = R.left
    
    # re-connect
    L.right = mid
    R.left = L 
    
    return R    # for telling the parent of L to re-connect to R

def rotate_right(self, R):
    
    # define the original parent-child relation
    L = R.left
    mid = L.right
    
    # re-connect
    R.left = mid
    L.right = R 
    
    return L    # for telling the parent of R to re-connect to L

<figure style="text-align: center">
    
    <figcaption> <b>Fig. 4</b> Links to be broken and reconnected in (a) <code>rotate_left()</code> over node L and (b) <code>rotate_right()</code> over node R.</figcaption>
</figure>

## 2.2. AVL tree

An AVL tree is a BST with the following additional rules:

- Every node has an extra property called `height`, which is the distance of itself from its furthest leaf descendent. E.g. a leaf node's `height` is 0, and its parent's is 1
- The `height` of the two children of a node can be different by at most 1.

**Consequences:** The path from a node to any leaves (`None` nodes) are at most different in height by 1. So the height of the tree always $\leq\log_2(n)$. Skewed tree never happens and it is very good for searching.

In [11]:
class Node:                        # re-define the node object
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.height = 0            # add a height property to each node


class AVL(BST):                         # Inherite from the BST class so I don't have to copy those container functions
    
    def __init__(self, root = None):    # declared via defining the root node
        self.root = root
        
        # functions that can be inherited from the BST class:
        # - print_tree
        # - insert_from_array
        # - search_data
        # - delete_data
        # - leftmost_descendent
        # insert() and delete() need to be re-written

    ###########################################################################
    # 2 new helping functions + modifying rotate_left(), rotate_right()
    
    # a function that returns a node's height
    def get_height(self, node):    
        if node is None:
            return -1              # define height of None node to be -1, so any height of any leaf node is 0
        return node.height
    
    
    # a function that returns the height difference in left/right subtree
    def get_balance(self, node):    
        if node is None:
            return 0
        return self.get_height(node.left) - self.get_height(node.right)
    
    
    def rotate_left(self, L):
        R = L.right
        mid = R.left

        L.right = mid
        R.left = L
        
        L.height = 1 + max(self.get_height(L.left), self.get_height(mid))              # need to update their heights after rotation
        R.height = 1 + max(self.get_height(R.right), L.height)                         # update the new child first, then the parent    
    
        return R

    def rotate_right(self, R):
        L = R.left
        mid = L.right

        R.left = mid
        L.right = R
            
        R.height = 1 + max(self.get_height(R.right), self.get_height(mid))             # need to update their heights after rotation
        L.height = 1 + max(self.get_height(L.left), R.height)                          # update the new child first, then the parent   

        return L
    

### 2.2.1 Insertion in AVL tree

Everytime after a node is inserted to the tree, the ancestor branch of the inserted node may become unbalanced (i.e. the two children of that node have `height` difference > 1). To fix this, traverse recursively from the inserted node to the first unbalanced node and do rotations. The result must be one of these four cases:

- Left-Right case
- Left-Left case
- Right-Left case
- Right-Right case

The procedures using `rotate_left()` and `rotate_right()` follow the picture below. 

<figure style="text-align: center">
    
    <figcaption> <b>Fig. 5</b> 4 cases in AVL tree and their fix-up steps.</figcaption>
</figure>

In [12]:
# the operations for fixing the AVL tree. It is the same for both insertion and deletion
def fix_AVL(self, node):

    # If (Left subtree height - Right subtree height) > 1, must be a Left-X case
    if self.get_balance(node) > 1:
        if self.get_balance(node.left) < 0:            # check the left child to see if it is a Left-Right case
            node.left = self.rotate_left(node.left)
        return self.rotate_right(node)                   # the new top-most node is returned

    # If (Left subtree height - Right subtree height) < -1, must be a Right-X case
    elif self.get_balance(node) < -1:
        if self.get_balance(node.right) > 0:            # check the right child to see if it is a Right-Left case
            node.right = self.rotate_right(node.right)
        return self.rotate_left(node)                    # the new top-most node is returned

    # if no fix needed, original node is still the topmost node
    return node


def insert(self, new_node, curr):      
    ###########################################################################
    # These are the same as insert in ordinary BST
    if curr is None:
        return new_node

    elif new_node.data < curr.data:
        curr.left = self.insert(new_node, curr.left)

    else:
        curr.right = self.insert(new_node, curr.right)

    # These are the same as insert in ordinary BST
    ###########################################################################

    # after previous recursion returns, 
    # 1. update the height of curr
    curr.height = 1 + max(self.get_height(curr.left), self.get_height(curr.right))

    # 2. see if curr's AVL property need to be fixed
    curr = self.fix_AVL(curr)

    return curr


AVL.fix_AVL = fix_AVL
AVL.insert = insert

In [13]:
tree = AVL()
tree.insert_from_array([20,11,29,5,17,24,33,1,8,27,19,14,28])
tree.print_tree('height')

[(20, 3)]
[(11, 2), (29, 2)]
[(5, 1), (17, 1), (27, 1), (33, 0)]
[(1, 0), (8, 0), (14, 0), (19, 0), (24, 0), (28, 0), None, None]


### 2.2.2. Deletion in AVL tree

The same 4 cases happen in deletion and the fixes are the same. Theoretically, 

- In insertion there could only be 1 unbalanced ancestor of the inserted node.
- In deletion there could result in more than 1 unbalanced ancestors. 

However in the coding we are doing recursion anyway and will check all ancestors of the changed nodes. Note that in the 2-children case, we choose the recursive method so the same height-fixing operations can be used the path from deleted node to its left-most descendent. 


In [14]:
def delete(self, target_value, curr):
    
    ###########################################################################
    # These are the same as delete in ordinary BST
    if curr is None:
        print('node not found')
        return None 
    
    # search for the node that contains the target value
    if target_value == curr.data:

        # case 1: curr has no child
        if curr.left is None and curr.right is None:
            return None

        
        # case 2: curr has 1 child
        if curr.right is None:
            return curr.left
        elif curr.left is None:
            return curr.right

        
        # case 3: curr has 2 children. This is the recursive method
        desc = self.leftmost_descendant(curr.right)
        desc.right = self.delete(desc.data, curr.right)                                   # recursive delete descendent from the branch
                                                                                          # then make it the new parent of the right branch
        desc.left = curr.left                                                             # also make it the new parent of the left branch
        
        desc.height = 1 + max(self.get_height(desc.left), self.get_height(desc.right))    # update descendent's height
        desc = self.fix_AVL(desc)
        
        return desc

    
    elif target_value < curr.data: 
        curr.left = self.delete(target_value, curr.left)

    else:
        curr.right = self.delete(target_value, curr.right)

    # These are the same as delete in ordinary BST
    ###########################################################################
    
    # after previous recursion returns, and if the node is found
    if curr:
        # 1. update the height of curr
        curr.height = 1 + max(self.get_height(curr.left), self.get_height(curr.right))

        # 2. see if curr's AVL property need to be fixed
        curr = self.fix_AVL(curr)

    return curr                                                 
  

AVL.delete = delete

In [15]:
tree = AVL()
a = [20,11,29,33,5,17,1,8,9]

tree.insert_from_array(a)
tree.print_tree('height')
print()
tree.delete_data(11)
tree.delete_data(17)
tree.print_tree('height')

[(20, 3)]
[(8, 2), (29, 1)]
[(5, 1), (11, 1), None, (33, 0)]
[(1, 0), None, (9, 0), (17, 0), None, None, None, None]

[(20, 3)]
[(8, 2), (29, 1)]
[(5, 1), (9, 0), None, (33, 0)]
[(1, 0), None, None, None, None, None, None, None]


## 2.3. Red-Black tree

A Red-Black tree (RB tree) is a BST with the following additional rules:

- Every node has an extra property called `color`, which can be either `red` or `black`.
- Root must be `black`.
- `None` nodes are considered as `black`.
- Children of a `red` node must be `black`.
- Any paths that travel from a node to any of its descendent `None` node must pass through the same number of `black` nodes. 

**Consequences:** The path from root to its furthest leaf node must be of the pattern black-red-black-red..., while the path to its closest leaf node must all be black nodes. So distance from furthest leaf is at most 2 times of the distance from the closest leaf. 

Although red-black tree is less balance than AVL tree, fixing it requires fewer rotation operations and node updates (In AVL, if the height changes, one must work all the way up back to root). Thus red-black tree is more efficient in insertion and deletion, even though the fixing rules are very annoying to remember.

In [16]:
class Node:                        # re-define the node object
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.color = True            # add a color property to each node. Denote red = True, black = False


class RBT(BST):                         # Inherite from the BST class so I don't have to copy those container functions
    
    def __init__(self, root = None):    # declared via defining the root node
        self.root = root
        
        # functions that can be inherited from the BST class:
        # - print_tree
        # - search_data
        # - delete_data
        # - leftmost_descendent
        # insert() and delete() need to be re-written

    ###########################################################################
    # 1 new helping functions + original rotate_left(), rotate_right()
    
    # a function that returns a node's color
    def get_color(self, node):    
        if node is None:
            return False              # None node has black color
        return node.color
    
    def rotate_left(self, L):
        R = L.right
        mid = R.left

        L.right = mid
        R.left = L    
    
        return R

    def rotate_right(self, R):
        L = R.left
        mid = L.right

        R.left = mid
        L.right = R   

        return L
    

### 2.3.1. Insertion in Red-Black tree

We always start with make the newly inserted node red. Call it the current node.

**The only possibility of violation is when its parent is also red.** (i.e. red's child = red)

The fixing procedures only concern these 4 nodes: **C** = Current, **P** = Parent, **U** = Uncle, **G** = Grandparent. Because the parent is red, the grandparent must be black. The uncle may be either black or red.

<figure style="text-align: center">

    <figcaption> <b>Fig. 6</b> Possibilities of violation in RBT insertion.</figcaption>
</figure>

The procedures of fixing are as follow: 

1. *(Case 0)* If the current node is the root, change it to black and it is done.
2. Check the parent. If its parent is black, it is done.

But if not, check the current node's uncle:

3. **(Case 1) If uncle is red, only need to re-color**.
<ol style="list-style-type: lower-roman">
    <li> Color its both parent and uncle as black
    <li> Color its grandparent as red, then call its grandparent the current node and repeat from step 1.
</ol><br>

    *Why these steps:* 
    - Re-coloring both parent and uncle to black already solve the red-red conflict. And this avoid the works of rotation. 
    - Need to color the grandparent to red so that the number of black nodes on the paths to descendents remains the same.
    - The red grandparent may lead to new red-red conflict, which shall be checked in the next recursion.

<figure style="text-align: center">

    <figcaption> <b>Fig. 7</b> Fixes for RBT insertion case 1.</figcaption>
</figure>

4. **(Case 2 & 3) If uncle is black, do rotation exactly like in AVL tree**.
<ol style="list-style-type: lower-roman">
    <li> <i>(Case 2)</i> If the grandparent-parent-current chain is in the L-R case  (or by mirroring, R-L case), do a rotation to become the L-L (or R-R) case.
    <li> <i>(Case 3)</i> Then at the L-L / R-R case, color the parent node to be black, and the new grandparent to be red.
    <li> Do the last rotation as in AVL tree to get a balanced tree.
</ol><br>
    
    *Why these steps:*
    - Unlike in case 1, making parent black will make the paths to descendents on parent's branch 1 black node more than the paths tp descendents on uncle's branch. Therefore rotation is the only option.
    - The fix in case 3 is equivalent to re-color the C-P-G-U chain to red-black-red-black, and then use the middle black node as the new grandparent.

<figure style="text-align: center">

    <figcaption> <b>Fig. 8</b> Fixes for RBT insertion case 2 & 3.</figcaption>
</figure>

In [17]:
def RBT_insert_fix(self, G, new_node):
    
    # Case 1: if uncle is red
    if new_node.data < G.data:
        U = G.right
    else:
        U = G.left
    if self.get_color(U):       
        G.left.color = False
        G.right.color = False
        G.color = True
        return G
    
    # Case 2 & 3 as in left branch of G
    if new_node.data < G.data:                           # Left-X case
        if new_node.data >= G.left.data:                 # Left-Right case need extra rotation (this is case 2)
            G.left = self.rotate_left(G.left)
        G.left.color = False                             # Then enter Left-Left case (this is case 3)
        G.color = True
        return self.rotate_right(G)
    
    # Case 2 & 3 as in right branch of G
    else:                                                # Right-X case
        if new_node.data < G.right.data:                 # Right-Left case need extra rotation (this is case 2)
            G.right = self.rotate_right(G.right)
        G.right.color = False                            # Then enter Right-Right case (this is case 3)
        G.color = True
        return self.rotate_left(G)
    

def insert_from_array(self, array):
    for data in array:
        self.root, _ = self.insert(Node(data), self.root)    # insertion always begin with the root
        
    self.root.color = False         ### case 0: always color the root as black at the end
    

def insert(self, new_node, curr):
    
    if curr is None:
        return new_node, False      # return the inserted node and color of its descendent on the path. 
                                    # For a node newly inserted, its child is always None and thus is black

    elif new_node.data < curr.data:
        curr.left, c = self.insert(new_node, curr.left)
        
        if self.get_color(curr.left) and c:         # treat curr as the grandparent. need to further check only if 
                                                    # the parent is red and the previous descendent is red
            
            curr = self.RBT_insert_fix(curr, new_node)      # it suffices to tell which configuration it is
                                                            # by the data of the inserted node
            
        return curr, self.get_color(curr.left)

    else:
        curr.right, c = self.insert(new_node, curr.right)
        
        if self.get_color(curr.right) and c:       # do the same check in another branch
            
            curr = self.RBT_insert_fix(curr, new_node)
            
        return curr, self.get_color(curr.right)



RBT.insert_from_array = insert_from_array
RBT.insert = insert
RBT.RBT_insert_fix = RBT_insert_fix

In [18]:
tree = RBT()
a = [20,11,29]
b = [33,5,17,1,8]

tree.insert_from_array(a)
tree.insert_from_array(b)
tree.print_tree('color')

[(20, False)]
[(11, True), (29, False)]
[(5, False), (17, False), None, (33, True)]
[(1, True), (8, True), None, None, None, None, None, None]


### 2.3.2. Deletion in Red-Black tree

**The only possibility of violation is when the deleted node is black**, because it can lead to the following:
- Along a path from a node to its descendent is missing 1 black node. Always happens unless the removed node is the root.
- Removing the black root and new root being replaced by a red node.
- A black node between two red nodes is removed, turning the red nodes to parent-child relation.

The procedures of fixing is a long story. We begin the analysis from the 3 cases in deletion.

1. Deleted node has no child - Need rotation fix only if deleted node is black.
2. Deleted node has 1 child
    - If deleted node is red, no violation. 
    - If deleted node is black and its child is red, just color its child to black.
    - Need rotation fix if deleted node and its child are both black.
3. Deleted node has 2 children 
    - In the recursive deletion, we start from deleting the left-most descendent which will be in case 1 or 2.
    - After returning and reconnection, color the left-most descendent to be the same color as the deleted node.

We can summarize the situation that needs a rotation fix as
> When a node is **removed** (the deleted node in no-child/1-child case, left-most descendent in 2-children case), 
> - **Itself is black.**
> - **Its only child (no matter a node or None) is also black.** 

The fixing procedures only concern these 5 nodes: **C** = Current, **P** = Parent, **S** = sibling, **N1/N2** = Nephews = sibling's children. The Parent, sibling and nephews may be red or black.

<figure style="text-align: center">

    <figcaption> <b>Fig. 9</b> Possibilities of violation in RBT deletion.</figcaption>
</figure>

The procedures of fixing are as follow:

1. (Case 0) If the removed node is the root, color the node that is going to replace it to black and it is done.
2. Check the removed node. If it is red, it is done.

But if not, check the child's sibling.

3. **(Case 1) Sibling is red.**
<ol style="list-style-type: lower-roman">
    <li> Color the parent (must be black originally) to red, and sibling to black.
    <li> Do rotation on sibling.
    <li> Since the nephews must also be black originally, this becomes one of the case 2, 3 or 4. At this point, current and its sibling must be both black, while the number of black nodes traversed is not yet fixed.
</ol>

<figure style="text-align: center">

    <figcaption> <b>Fig. 10</b> Fixes for RBT deletion case 1.</figcaption>
</figure>

4. **(Case 2) Sibling is black, and both nephews are black.**
<ol style="list-style-type: lower-roman">
    <li> Color the sibling to red
    <li> If parent is red or is the root, color it to black. The number of black nodes traversed is fixed.
    <li> If parent is black, need to fix the number of black nodes in the next recursion. 
</ol>

<figure style="text-align: center">

    <figcaption> <b>Fig. 11</b> Fixes for RBT deletion case 2.</figcaption>
</figure>

5. **(Case 3 & 4) Sibling is black, and at least 1 nephew is red.** <br>
    *(Case 3)* There is only 1 red nephew and the parent-sibling-red nephew chain is in the R-L case (or by mirroring, L-R case)
    <ol style="list-style-type: lower-roman">
        <li> Color the red nephew to black and sibling to red.
        <li> Do a rotation on the sibling to make the parent-sibling-red nephew chain into the R-R case (or L-L case).
    </ol><br>
        
    *(Case 4)* There is a parent-sibling-red nephew chain in the R-R / L-L case, 
    <ol style="list-style-type: lower-roman">
        <li> Color sibling to parent's color, then parent to black.
        <li> Color the red nephew on the R-R / L-L chain to black. Ignore the other nephew (whether red or not).
        <li> Do a rotation on the sibling.
    </ol>

<figure style="text-align: center">

    <figcaption> <b>Fig. 12</b> Fixes for RBT deletion case 3 & 4.</figcaption>
</figure>

In [19]:
def RBT_delete_fix(self, parent, curr):
    
    # curr on parent's left branch
    if curr == parent.left: 
        
        # case 1: sibling is red
        if self.get_color(parent.right):
            parent.color = True
            parent.right.color = False
            S = self.rotate_left(parent)                     # the return is the S node in the figure
            S.left, _ = self.RBT_delete_fix(parent, curr)    # call delete_fix again which will enter case 2, 3 or 4
                                                             # note that P is red. if it goes into case 2, the returned tree is guarenteed
                                                             # to be fixed. No need to do another layer of recursion.
            return S, False
        
        # case 2: sibling is black and both nephews are black
        elif not (self.get_color(parent.right.right) or self.get_color(parent.right.left)):
            parent.right.color = True
            if parent.color:
                parent.color = False
                return parent, False                      # color P to black if it is originally red.
            return parent, True                           # need to run the fix again in the next recursion if P is already black
        
        # case 3 & 4
        # case 3: right-right nephew is not red (N2)
        elif not self.get_color(parent.right.right):
            parent.right.left.color = False
            parent.right.color = True
            parent.right = self.rotate_right(parent.right)
        # case 4: right-right nephew is red
        parent.right.color = parent.color
        parent.color = False
        parent.right.right.color = False
        return self.rotate_left(parent), False
            

    # curr on parent's left branch. Copy and paste and mirror the whole thing above
    else:
        
        # case 1: sibling is red
        if self.get_color(parent.left):
            parent.color = True
            parent.left.color = False
            S = self.rotate_right(parent)
            S.right, _ = self.RBT_delete_fix(parent, curr)
            return S, False
        
        # case 2: sibling is black and both nephews are black
        elif not (self.get_color(parent.left.left) or self.get_color(parent.left.right)):
            parent.left.color = True
            if parent.color:
                parent.color = False
                return parent, False
            return parent, True
        
        # case 3 & 4
        # case 3: left-left nephew is not red (N2)
        elif not self.get_color(parent.left.left):
            parent.left.right.color = False
            parent.left.color = True
            parent.left = self.rotate_left(parent.left)
        # case 4: left-left nephew is red
        parent.left.color = parent.color
        parent.color = False
        parent.left.left.color = False
        return self.rotate_right(parent), False

    
def delete_data(self, target_value):
    self.root, _ = self.delete(target_value, self.root)
    
    self.root.color = False         ### case 0: always color the root as black at the end
    
     
def delete(self, target_value, curr):
    
    if curr is None:
        print('node not found')
        return None, False
    
    # search for the node that contains the target value
    if target_value == curr.data:

        # The return contain 1. the new node to reconnect (same as original BST); 
        #                    2. A True/False flag "need_fix" to see if it needs to run RBT fix
        
        # case 1: curr has no child
        if curr.left is None and curr.right is None:
            return None, not curr.color                 # Deleted node = Red = True = not need to fix -> flag = False
                                                        # Deleted node = Black = False = need to fix -> flag = True

        # case 2: curr has 1 child
        if curr.right is None:
            if curr.left.color:                         # Deleted node's only child is red -> just color it to black
                curr.left.color = False                 # no more other fix
                return curr.left, False
            return curr.left, not curr.color
        
        elif curr.left is None:
            if curr.right.color:
                curr.right.color = False
                return curr.right, False
            return curr.right, not curr.color

        
        # case 3: curr has 2 children. This is the recursive method
        desc = self.leftmost_descendant(curr.right)
        desc.right, need_fix = self.delete(desc.data, curr.right)
        desc.left = curr.left
        
        if need_fix:
            desc, need_fix = self.RBT_delete_fix(desc, desc.right)
        
        return desc, need_fix

    
    # after the previous recursion return, check if the node need further fix
    elif target_value < curr.data: 
        curr.left, need_fix = self.delete(target_value, curr.left)
        
        if need_fix:
            curr, need_fix = self.RBT_delete_fix(curr, curr.left)

    else:
        curr.right, need_fix = self.delete(target_value, curr.right)
        
        if need_fix:
            curr, need_fix = self.RBT_delete_fix(curr, curr.right)
        

    return curr, need_fix
  

RBT.delete = delete
RBT.delete_data = delete_data
RBT.RBT_delete_fix = RBT_delete_fix

In [20]:
tree = RBT()
a = [55,40,65,60,75,57]

tree.insert_from_array(a)
tree.print_tree('color')
print()
tree.delete_data(40)
tree.print_tree('color')

[(55, False)]
[(40, False), (65, True)]
[None, None, (60, False), (75, False)]
[None, None, None, None, (57, True), None, None, None]

[(65, False)]
[(57, True), (75, False)]
[(55, False), (60, False), None, None]


---
# 3. Practice

## 3.1. Fibonacci tree

A Fibonacci tree can be considered the worst case AVL tree in that it has the smallest number of nodes given the tree height $h$. Draw Fibonacci trees for $h = 1,2,3,4$ and justify the name of the tree. (P.S. tree with $h=0$ only contains the root node)

## 3.2. Divide a binary search tree

Write an algorithm that divide a BST into two trees - one tree with `.data` $< k$ and the other tree with `.data` $\geq k$, where $k$ equals to the `.data` of one of the node.  

In [ ]:
# definition of Node and BST are copied from section 1
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        
class BST:
    def __init__(self, root = None):
        self.root = root

    ###########################################################################
    # try it yourself
    def divide_into_2(self, k):
    
    